In [1]:
# Preprocessing
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Modelling
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor, Pool
from timeit import default_timer as timer
import optuna

%matplotlib inline
plt.style.use("bmh")

path = Path.cwd().parent / "mdsb-2023"

In [2]:
def train_test_split_temporal(X, y, delta_threshold="60 days"):

    cutoff_date = X["date"].max() - pd.Timedelta(delta_threshold)
    mask = X["date"] <= cutoff_date
    X_train, X_valid = X.loc[mask], X.loc[~mask]
    y_train, y_valid = y[mask], y[~mask]

    return X_train, y_train, X_valid, y_valid

### Define pipeline functions

In [3]:
def _encode_dates(X, col_name="date"):
    X = X.copy()

    X["month"] = X[col_name].dt.month
    X["weekday"] = X[col_name].dt.weekday
    X["hour"] = X[col_name].dt.hour

    X["month_sin"] = np.sin(2 * np.pi * X["date"].dt.month / 12)
    X["month_cos"] = np.cos(2 * np.pi * X["date"].dt.month / 12)

    X["day_sin"] = np.sin(2 * np.pi * X["date"].dt.day / X["date"].dt.days_in_month)
    X["day_cos"] = np.cos(2 * np.pi * X["date"].dt.day / X["date"].dt.days_in_month)

    X["hour_sin"] = np.sin(2 * np.pi * X["date"].dt.hour / 24)
    X["hour_cos"] = np.cos(2 * np.pi * X["date"].dt.hour / 24)

    X[["month", "weekday", "hour"]] = X[["month", "weekday", "hour"]].astype("category")

    return X.drop(columns=[col_name])


def _encode_covid(X, col_name="date"):
    X = X.copy()

    # Create masks for lockdown dates
    lockdown_1 = (X["date"] >= "2020-10-17") & (X["date"] <= "2020-12-14")

    lockdown_2 = (X["date"] >= "2020-12-15") & (X["date"] <= "2021-02-26")

    lockdown_3 = (X["date"] >= "2021-02-27") & (X["date"] <= "2021-05-02")

    X["Covid"] = 0
    X.loc[lockdown_1 | lockdown_2 | lockdown_3, "Covid"] = 1

    return X


def _merge_external_data(X, include_lags=True, include_ma=True):
    to_keep = [
        "date",
        "hnuage4",
        "t",
        "ctype4",
        "nnuage4",
        "u",
        "etat_sol",
        "perssfrai",
        "tx12",
        "cm",
        "tn12",
        "tend24",
        "vv",
        "rafper",
        "rr24",
        "hnuage2",
        "td",
        "rr3",
        "hnuage3",
        "hnuage1",
    ]

    ext_data = pd.read_csv(path / "external_data.csv", parse_dates=["date"])[to_keep]

    ext_data.drop(columns=ext_data.columns[ext_data.isna().sum() > 1000], inplace=True)

    full_date_range = pd.date_range(
        start=np.min([np.min(data.date), np.min(test.date)]),
        end=np.max([np.max(data.date), np.max(test.date)]),
        freq="H",
    )

    full_date_range = pd.DataFrame({"date": full_date_range})

    ext_data = full_date_range.merge(ext_data, on="date", how="left")

    columns_to_interpolate = ext_data.drop(columns="date").columns
    ext_data[columns_to_interpolate] = (
        ext_data[columns_to_interpolate]
        .interpolate(method="polynomial", order=3)
        .interpolate(method="bfill")
        .interpolate(method="ffill")
    )

    X = X.copy()

    X["date"] = X["date"].astype("datetime64[ns]")
    ext_data["date"] = ext_data["date"].astype("datetime64[ns]")

    X["orig_index"] = np.arange(X.shape[0])

    X = pd.merge_asof(X.sort_values("date"), ext_data.sort_values("date"), on="date")

    # Sort back to the original order
    X = X.sort_values("orig_index")
    del X["orig_index"]

    return X


def _gas_price_encoder(X):
    X = X.copy()
    X["gas_price"] = 1

    gas_prices = np.array(
        [
            1.22,
            1.21,
            1.22,
            1.27,
            1.31,
            1.36,
            1.4,
            1.39,
            1.4,
            1.43,
            1.45,
            1.45,
            1.46,
            1.56,
        ]
    )

    years = [
        2020,
        2020,
        2020,
        2020,
        2021,
        2021,
        2021,
        2021,
        2021,
        2021,
        2021,
        2021,
        2021,
        2021,
    ]

    months = [9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

    for i, price in enumerate(gas_prices):
        X.loc[
            (X.date.dt.month == months[i]) & (X.date.dt.year == years[i]), "gas_price"
        ] = price

    return X


def full_encode(X):
    return _encode_dates(_encode_covid(_gas_price_encoder(_merge_external_data(X))))

## Import main dataset

In [4]:
data = pd.read_parquet(path / "train.parquet")
test = pd.read_parquet(path / "final_test.parquet")

targets = ["bike_count", "log_bike_count"]

In [5]:
data.drop(
    columns=[
        "site_name",
        "counter_id",
        "site_id",
        "counter_installation_date",
        "coordinates",
        "counter_technical_id",
    ],
    inplace=True,
)

## Optuna def

In [6]:
def objective(trial):
    # Define hyperparameters to optimize
    learning_rate = trial.suggest_float("learning_rate", 0.1, 0.25)
    max_depth = trial.suggest_int("max_depth", 5, 9)
    n_estimators = trial.suggest_int("n_estimators", 250, 1000)
    subsample = trial.suggest_float("subsample", 0.7, 0.95)

    data_merger = FunctionTransformer(_merge_external_data, validate=False)
    covid_encoder = FunctionTransformer(_encode_covid, validate=False)
    gas_encoder = FunctionTransformer(_gas_price_encoder, validate=False)
    date_encoder = FunctionTransformer(_encode_dates, validate=False)

    regressor = CatBoostRegressor(
        learning_rate=learning_rate,
        max_depth=max_depth,
        n_estimators=n_estimators,
        subsample=subsample,
        od_pval=1e-5,
    )

    pipe = Pipeline(
        [
            ("merge external", data_merger),
            ("gas prices encoder", gas_encoder),
            ("covid encoder", covid_encoder),
            ("date encoder", date_encoder),
            ("regressor", regressor),
        ]
    )

    # Perform temporal train-test split
    X_train, y_train, X_valid, y_valid = train_test_split_temporal(X, y)

    val_pool = Pool(
        full_encode(X_valid),
        label=y_valid,
        cat_features=categorical_cols,
    )

    # Fit the pipeline on the training data
    pipe.fit(
        X_train,
        y_train,
        regressor__cat_features=categorical_cols,
        regressor__early_stopping_rounds=130,
        regressor__eval_set=val_pool,
    )

    # Make predictions on the validation set
    predictions = pipe.predict(X_valid)

    # Calculate the mean squared error as the objective
    neg_rmse = -mean_squared_error(y_valid, predictions, squared=False)

    return neg_rmse

## Model

In [7]:
X, y = data.drop(columns=targets), data["log_bike_count"]

In [8]:
date_cols = (
    _encode_dates(X[["date"]]).select_dtypes(include="category").columns.tolist()
)
categorical_cols = ["counter_name"] + date_cols

In [9]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

# Get the best parameters from the study
best_params = study.best_params
print("Best Hyperparameters:", best_params)

[I 2023-12-17 19:10:18,380] A new study created in memory with name: no-name-ebc4be80-5f15-47e1-8cf4-647eb572ea36


0:	learn: 1.4263605	test: 1.2223587	best: 1.2223587 (0)	total: 561ms	remaining: 2m 31s
1:	learn: 1.2209971	test: 1.0396607	best: 1.0396607 (1)	total: 911ms	remaining: 2m 2s
2:	learn: 1.0723665	test: 0.9193003	best: 0.9193003 (2)	total: 1.14s	remaining: 1m 41s
3:	learn: 0.9676072	test: 0.8265461	best: 0.8265461 (3)	total: 1.4s	remaining: 1m 33s
4:	learn: 0.8839855	test: 0.7863990	best: 0.7863990 (4)	total: 1.64s	remaining: 1m 27s
5:	learn: 0.8262808	test: 0.7537143	best: 0.7537143 (5)	total: 1.94s	remaining: 1m 25s
6:	learn: 0.7641257	test: 0.7336851	best: 0.7336851 (6)	total: 2.26s	remaining: 1m 25s
7:	learn: 0.7201274	test: 0.7307000	best: 0.7307000 (7)	total: 2.56s	remaining: 1m 24s
8:	learn: 0.6878349	test: 0.7196627	best: 0.7196627 (8)	total: 2.79s	remaining: 1m 21s
9:	learn: 0.6620691	test: 0.7023425	best: 0.7023425 (9)	total: 3.03s	remaining: 1m 19s
10:	learn: 0.6435102	test: 0.7009005	best: 0.7009005 (10)	total: 3.38s	remaining: 1m 19s
11:	learn: 0.6306790	test: 0.7028837	best: 

[I 2023-12-17 19:11:48,486] Trial 0 finished with value: -0.6202298027479825 and parameters: {'learning_rate': 0.22338386068911087, 'max_depth': 8, 'n_estimators': 271, 'subsample': 0.9428775710400238}. Best is trial 0 with value: -0.6202298027479825.


0:	learn: 1.4871642	test: 1.2773318	best: 1.2773318 (0)	total: 438ms	remaining: 2m 58s
1:	learn: 1.3166091	test: 1.1155756	best: 1.1155756 (1)	total: 695ms	remaining: 2m 21s
2:	learn: 1.1804204	test: 0.9889852	best: 0.9889852 (2)	total: 909ms	remaining: 2m 3s
3:	learn: 1.0630866	test: 0.9213170	best: 0.9213170 (3)	total: 1.27s	remaining: 2m 8s
4:	learn: 0.9748082	test: 0.8497356	best: 0.8497356 (4)	total: 1.52s	remaining: 2m 3s
5:	learn: 0.9056966	test: 0.7957421	best: 0.7957421 (5)	total: 1.96s	remaining: 2m 11s
6:	learn: 0.8471374	test: 0.7769606	best: 0.7769606 (6)	total: 2.22s	remaining: 2m 7s
7:	learn: 0.8014885	test: 0.7493438	best: 0.7493438 (7)	total: 2.5s	remaining: 2m 5s
8:	learn: 0.7522587	test: 0.7310170	best: 0.7310170 (8)	total: 2.87s	remaining: 2m 7s
9:	learn: 0.7153540	test: 0.7229071	best: 0.7229071 (9)	total: 3.18s	remaining: 2m 7s
10:	learn: 0.6875051	test: 0.7186081	best: 0.7186081 (10)	total: 3.4s	remaining: 2m 3s
11:	learn: 0.6674249	test: 0.7189111	best: 0.718608

[I 2023-12-17 19:13:29,556] Trial 1 finished with value: -0.6601027808437778 and parameters: {'learning_rate': 0.16750555608392687, 'max_depth': 9, 'n_estimators': 409, 'subsample': 0.7554527325319703}. Best is trial 0 with value: -0.6202298027479825.


...

0:	learn: 1.5020227	test: 1.2958608	best: 1.2958608 (0)	total: 1.03s	remaining: 4m 25s
1:	learn: 1.3433323	test: 1.1469702	best: 1.1469702 (1)	total: 1.58s	remaining: 3m 22s
2:	learn: 1.2186306	test: 1.0377736	best: 1.0377736 (2)	total: 2.11s	remaining: 3m
3:	learn: 1.1168343	test: 0.9528264	best: 0.9528264 (3)	total: 2.84s	remaining: 3m 1s
4:	learn: 1.0389004	test: 0.8823009	best: 0.8823009 (4)	total: 3.2s	remaining: 2m 42s
5:	learn: 0.9748714	test: 0.8339580	best: 0.8339580 (5)	total: 3.44s	remaining: 2m 25s
6:	learn: 0.9262326	test: 0.8035136	best: 0.8035136 (6)	total: 3.98s	remaining: 2m 23s
7:	learn: 0.8851942	test: 0.7721274	best: 0.7721274 (7)	total: 4.29s	remaining: 2m 14s
8:	learn: 0.8525174	test: 0.7550478	best: 0.7550478 (8)	total: 4.89s	remaining: 2m 15s
9:	learn: 0.8099127	test: 0.7397525	best: 0.7397525 (9)	total: 5.53s	remaining: 2m 17s
10:	learn: 0.7767248	test: 0.7250437	best: 0.7250437 (10)	total: 6.16s	remaining: 2m 18s
11:	learn: 0.7516333	test: 0.7216506	best: 0.72

[I 2023-12-17 21:09:18,103] Trial 28 finished with value: -0.6157287508988727 and parameters: {'learning_rate': 0.16841693526516655, 'max_depth': 6, 'n_estimators': 259, 'subsample': 0.9276600529696647}. Best is trial 19 with value: -0.6077230859726632.


0:	learn: 1.4988863	test: 1.2930706	best: 1.2930706 (0)	total: 1.04s	remaining: 12m 40s
1:	learn: 1.3383052	test: 1.1424481	best: 1.1424481 (1)	total: 1.68s	remaining: 10m 13s
2:	learn: 1.2127344	test: 1.0326788	best: 1.0326788 (2)	total: 2.18s	remaining: 8m 49s
3:	learn: 1.1104550	test: 0.9475483	best: 0.9475483 (3)	total: 2.97s	remaining: 9m
4:	learn: 1.0326737	test: 0.8773499	best: 0.8773499 (4)	total: 3.55s	remaining: 8m 35s
5:	learn: 0.9690404	test: 0.8295961	best: 0.8295961 (5)	total: 3.96s	remaining: 7m 58s
6:	learn: 0.9203724	test: 0.7950320	best: 0.7950320 (6)	total: 4.72s	remaining: 8m 8s
7:	learn: 0.8813255	test: 0.7701090	best: 0.7701090 (7)	total: 5.34s	remaining: 8m 3s
8:	learn: 0.8399155	test: 0.7661941	best: 0.7661941 (8)	total: 6.03s	remaining: 8m 4s
9:	learn: 0.8124659	test: 0.7485037	best: 0.7485037 (9)	total: 6.69s	remaining: 8m 3s
10:	learn: 0.7890446	test: 0.7498518	best: 0.7485037 (9)	total: 7.43s	remaining: 8m 7s
11:	learn: 0.7553844	test: 0.7404362	best: 0.7404

[I 2023-12-17 21:16:03,045] Trial 29 finished with value: -0.6483848176335316 and parameters: {'learning_rate': 0.171248680544105, 'max_depth': 6, 'n_estimators': 732, 'subsample': 0.9248324704362063}. Best is trial 19 with value: -0.6077230859726632.


Best Hyperparameters: {'learning_rate': 0.18671203765453545, 'max_depth': 6, 'n_estimators': 774, 'subsample': 0.8239955828932404}


In [10]:
best_params

{'learning_rate': 0.18671203765453545,
 'max_depth': 6,
 'n_estimators': 774,
 'subsample': 0.8239955828932404}